In [ ]:
import torch

# 预测部分

In [ ]:
def decoder(pred):
    '''
    输入：(1x7x7x30)---[x1,y1,x2,y2,置信度1,x1,y1,x2,y2,置信度1,....]
    输出：
    '''
    grid_num = 14
    cell_size = 1./grid_num
    pred = pred.data
    # 去掉batch_size维度--->(7x7x30)
    pred = pred.squeeze(0)
    # (7x7x1)
    contain1 = pred[:, :, 4].unsqueeze(2)
    # (7x7x1)
    contain2 = pred[:, :, 9].unsqueeze(2)
    # (7x7x2)
    contain = torch.cat((contain1, contain2), 2)
    # 把（7x7x2）里的数值替换为true, false
    mask1 = contain > 0.1
    mask2 = (contain == contain.max())
    # 将mask1,mask2进行 或 操作
    mask = (mask1 + mask2).gt(0)
    
    for i in range(grid_num):
        for j in range(grid_num):
            for b in range(2):
                if mask[i, j, b] == 1:
                    box = pred[i, j, b*5:b*5+4]
                    contain_prob = torch.FloatTensor([pred[i, j, b*5+4]])
                    ## 转换坐标
                    xy = torch.FloatTensor([j, i]) * cell_size
                    box[:2] = box[:2] * cell_size + xy
                    
                    max_prob, cls_index = torch.max(pred[i, j, 10:], 0)
                    if float((contain_prob * max_prob)[0]) > 0.1:
                        ## 添加